<a href="https://colab.research.google.com/github/faisal-patwary/Hate-Crime-Tristate-Data/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import pandas as pd
import os

In [80]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
os.listdir('/content/drive/MyDrive/CIS4400_HateCrimes')

['hate_crime_raw.csv',
 'hate_crime_raw.gsheet',
 'hate_crimes_dictionary.gsheet']

In [82]:
filepath = '/content/drive/MyDrive/CIS4400_HateCrimes/hate_crime_raw.csv'
df = pd.read_csv(filepath)
df.head()

,incident_id,data_year,ori,pug_agency_name,pub_agency_unit,agency_type_name,state_abbr,state_name,division_name,region_name,population_group_code,population_group_description,incident_date,adult_victim_count,juvenile_victim_count,total_offender_count,adult_offender_count,juvenile_offender_count,offender_race,offender_ethnicity,victim_count,offense_name,total_individual_victims,location_name,bias_desc,victim_types,multiple_offense,multiple_bias
0,43,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,3,"Cities from 50,000 thru 99,999",7/4/1991,NaN,NaN,1,NaN,NaN,Black or African American,Not Specified,1,Aggravated Assault,1.0,Residence/Home,Anti-Black or African American,Individual,S,S
1,44,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,3,"Cities from 50,000 thru 99,999",12/24/1991,NaN,NaN,1,NaN,NaN,Black or African American,Not Specified,2,Aggravated Assault;Destruction/Damage/Vandalism of Property,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,M,S
2,45,1991,AR0600300,North Little Rock,NaN,City,AR,Arkansas,West South Central,South,3,"Cities from 50,000 thru 99,999",7/10/1991,NaN,NaN,1,NaN,NaN,Black or African American,Not Specified,2,Aggravated Assault;Murder and Nonnegligent Manslaughter,2.0,Residence/Home,Anti-White,Individual,M,S
3,46,1991,AR0600300,North Little Rock,NaN,City,AR,Arkansas,West South Central,South,3,"Cities from 50,000 thru 99,999",10/6/1991,NaN,NaN,2,NaN,NaN,Black or African American,Not Specified,1,Intimidation,1.0,Residence/Home,Anti-White,Individual,S,S
4,47,1991,AR0670000,Sevier,NaN,County,AR,Arkansas,West South Central,South,8D,"Non-MSA counties under 10,000",10/14/1991,NaN,NaN,1,NaN,NaN,White,Not Specified,1,Intimidation,1.0,School/College,Anti-Black or African American,Individual,S,S


In [83]:
df.columns

Index(['incident_id', 'data_year', 'ori', 'pug_agency_name', 'pub_agency_unit',
       'agency_type_name', 'state_abbr', 'state_name', 'division_name',
       'region_name', 'population_group_code', 'population_group_description',
       'incident_date', 'adult_victim_count', 'juvenile_victim_count',
       'total_offender_count', 'adult_offender_count',
       'juvenile_offender_count', 'offender_race', 'offender_ethnicity',
       'victim_count', 'offense_name', 'total_individual_victims',
       'location_name', 'bias_desc', 'victim_types', 'multiple_offense',
       'multiple_bias'],
      dtype='object')

In [84]:
df.shape

(253776, 28)

In [85]:
# filter for data_year 2020 and later
df = df[df['data_year'] >= 2020]

# split the two columns on “/” into lists
df['offense_name']  = df['offense_name'].str.split('/')
df['location_name'] = df['location_name'].str.split('/')

# explode both lists so each value gets its own row
df = df.explode('offense_name').explode('location_name').reset_index(drop=True)

# drop the specified columns
df = df.drop(columns=['pub_agency_unit', 'division_name', 'region_name'])

# drop NaN values
df = df.dropna()

# convert those columns to integers
float_cols = df.select_dtypes(include=['float']).columns
df[float_cols] = df[float_cols].astype(int)

# check row count
print(f"Rows after cleaning: {len(df)}")

Rows after cleaning: 102840


In [86]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


df.head(10)

,incident_id,data_year,ori,pug_agency_name,agency_type_name,state_abbr,state_name,population_group_code,population_group_description,incident_date,adult_victim_count,juvenile_victim_count,total_offender_count,adult_offender_count,juvenile_offender_count,offender_race,offender_ethnicity,victim_count,offense_name,total_individual_victims,location_name,bias_desc,victim_types,multiple_offense,multiple_bias
8,893434,2020,AK0010200,Fairbanks,City,AK,Alaska,4,"Cities from 25,000 thru 49,999",1/3/2020,1,0,1,1,0,White,Not Hispanic or Latino,1,Aggravated Assault,1,Government,Anti-Transgender,Individual,S,S
9,893434,2020,AK0010200,Fairbanks,City,AK,Alaska,4,"Cities from 25,000 thru 49,999",1/3/2020,1,0,1,1,0,White,Not Hispanic or Latino,1,Aggravated Assault,1,Public Building,Anti-Transgender,Individual,S,S
10,1052479,2020,AK0010300,Juneau,City,AK,Alaska,4,"Cities from 25,000 thru 49,999",5/4/2020,1,0,0,0,0,Unknown,Unknown,1,Aggravated Assault,1,Parking,Anti-American Indian or Alaska Native,Individual,S,S
11,1052479,2020,AK0010300,Juneau,City,AK,Alaska,4,"Cities from 25,000 thru 49,999",5/4/2020,1,0,0,0,0,Unknown,Unknown,1,Aggravated Assault,1,Drop Lot,Anti-American Indian or Alaska Native,Individual,S,S
12,1052479,2020,AK0010300,Juneau,City,AK,Alaska,4,"Cities from 25,000 thru 49,999",5/4/2020,1,0,0,0,0,Unknown,Unknown,1,Aggravated Assault,1,Garage,Anti-American Indian or Alaska Native,Individual,S,S
13,1054279,2020,AK0010100,Anchorage,City,AK,Alaska,1C,"Cities from 250,000 thru 499,999",6/1/2020,1,0,1,1,0,Unknown,Unknown,1,Sexual Assault With An Object,1,Park,Anti-Female,Individual,S,S
14,1054279,2020,AK0010100,Anchorage,City,AK,Alaska,1C,"Cities from 250,000 thru 499,999",6/1/2020,1,0,1,1,0,Unknown,Unknown,1,Sexual Assault With An Object,1,Playground,Anti-Female,Individual,S,S
15,1054280,2020,AK0010100,Anchorage,City,AK,Alaska,1C,"Cities from 250,000 thru 499,999",6/21/2020,1,0,1,1,0,American Indian or Alaska Native,Not Hispanic or Latino,1,Aggravated Assault,1,Hotel,Anti-Black or African American,Individual,S,S
16,1054280,2020,AK0010100,Anchorage,City,AK,Alaska,1C,"Cities from 250,000 thru 499,999",6/21/2020,1,0,1,1,0,American Indian or Alaska Native,Not Hispanic or Latino,1,Aggravated Assault,1,Motel,Anti-Black or African American,Individual,S,S
17,1054280,2020,AK0010100,Anchorage,City,AK,Alaska,1C,"Cities from 250,000 thru 499,999",6/21/2020,1,0,1,1,0,American Indian or Alaska Native,Not Hispanic or Latino,1,Aggravated Assault,1,Etc.,Anti-Black or African American,Individual,S,S


In [87]:
print(len(df))

102840


In [88]:
df.isnull().sum()

,0
incident_id,0
data_year,0
ori,0
pug_agency_name,0
agency_type_name,0
state_abbr,0
state_name,0
population_group_code,0
population_group_description,0
incident_date,0


In [ ]:
df.to_csv('hate_crime_cleaned.csv', index = False)
print('Complete')

Complete
